In [1]:
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from matplotlib import gridspec

from sklearn.preprocessing import scale

from tqdm.notebook import tqdm

import json

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

from torch.utils.data import Dataset

from sklearn.utils import shuffle
from scipy.special import softmax

from PIL import Image

from PIL import ImageDraw

from skimage import morphology

import pickle

from sklearn.model_selection import ParameterGrid

In [2]:
from mixturelib.mixture import MixtureEM
from mixturelib.local_models import EachModelLinear, EachModel
from mixturelib.hyper_models import HyperExpertNN, HyperModelDirichlet

from mixturelib.regularizers import RegularizeModel

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [7]:
def get_data(beta=0.1, l=200):
    np.random.seed(0)

    t = np.linspace(-2, 2, l)

    p = 0.5
    alpha = 0.9
    x0 = 0
    y0 = 0

    x=x1=((t**2)/(2*p))*np.cos(alpha)-t*np.sin(alpha)+x0
    y=y1=((t**2)/(2*p))*np.sin(alpha)+t*np.cos(alpha)+y0
    X = np.vstack([x*y, y**2, x, y, np.ones_like(x)]).T
    Y = x**2
    w1 = np.linalg.inv(X.T@X)@X.T@Y
    x1 += beta*np.random.randn(l)
    y1 += beta*np.random.randn(l)


    p = 0.3
    alpha = 2
    x0 = 5
    y0 = 5
    x=x2=((t**2)/(2*p))*np.cos(alpha)-t*np.sin(alpha)+x0
    y=y2=((t**2)/(2*p))*np.sin(alpha)+t*np.cos(alpha)+y0
    X = np.vstack([x*y, y**2, x, y, np.ones_like(x)]).T
    Y = x**2
    w2 = np.linalg.inv(X.T@X)@X.T@Y
    x2 += beta*np.random.randn(l)
    y2 += beta*np.random.randn(l)


    x = np.hstack([x1, x2])
    y = np.hstack([y1, y2])
    
    return x, y, w1, w2

In [8]:
x, y, w1, w2 = get_data(beta=0.1)

In [9]:
X = np.vstack([x*y, y**2, x, y, np.ones_like(x)]).T
Y = x**2

In [10]:
X_tr = torch.FloatTensor(X).to(device)
Y_tr = torch.FloatTensor(Y).view([-1,1]).to(device)